In [ ]:
import parsy

In [ ]:
auto_pick_output_text = r"""INFO:root:native_project.Name='frankNstein_Bakken_UTM13_FEET'
INFO:root:len(native_project.ObservationSets.Items)=2
INFO:root:observation_set.Name='ParentWellObservations'
INFO:root:len(observation_set.GetObservations())=17
INFO:root:observation_set.Name='Auto-picked Observation Set3'
INFO:root:len(observation_set.GetObservations())=120
INFO:root:Wrote changes to "c:\src\Orchid.IntegrationTestData\frankNstein_Bakken_UTM13_FEET.999.ifrac"
"""

In [ ]:
auto_pick_output_text

In [ ]:
# Utility parsers
newline = parsy.string('\n')
left_brace = parsy.string('{')
right_brace = parsy.string('}')
integer = parsy.regex(r'\d+').map(int)

In [ ]:
# Single-line parsers
project_name = parsy.string("INFO:root:native_project.Name='frankNstein_Bakken_UTM13_FEET'")
observation_set_items = parsy.string("INFO:root:len(native_project.ObservationSets.Items)=2")
parent_well_observations = parsy.string("INFO:root:observation_set.Name='ParentWellObservations'")
get_observations = parsy.string("INFO:root:len(observation_set.GetObservations())=") >> parsy.regex(r'\d+').map(int)
auto_picked_observation_set = parsy.string("INFO:root:observation_set.Name='Auto-picked Observation Set3'")
output_path_name = (parsy.string('INFO:root:Wrote changes to') >>
                    parsy.regex(r' "c:\\src\\Orchid.IntegrationTestData\\frankNstein_Bakken_UTM13_FEET.\d{3}.ifrac"'))
unique_attribute_count_per_stage_per_well_equals = parsy.string(r'INFO:root:Unique counts of attributes per stage per well=')

In [ ]:
@parsy.generate
def attribute_count_per_stage_per_well():
    yield unique_attribute_count_per_stage_per_well_equals
    yield left_brace
    attribute_per_stage_per_well_count = yield integer
    yield right_brace
    
    return attribute_per_stage_per_well_count

In [ ]:
@parsy.generate
def get_second_observations_count():
    yield project_name
    yield newline >> observation_set_items
    yield newline >> parent_well_observations
    yield newline >> get_observations
    yield newline >> auto_picked_observation_set
    get_observations_count = yield newline >> get_observations
    yield (newline >> attribute_count_per_stage_per_well).optional()
    yield newline >> output_path_name
    yield newline

    return get_observations_count

In [ ]:
get_second_observations_count.parse(auto_pick_output_text)

In [ ]:
auto_pick_and_create_attributes_output_text = r"""INFO:root:native_project.Name='frankNstein_Bakken_UTM13_FEET'
INFO:root:len(native_project.ObservationSets.Items)=2
INFO:root:observation_set.Name='ParentWellObservations'
INFO:root:len(observation_set.GetObservations())=17
INFO:root:observation_set.Name='Auto-picked Observation Set3'
INFO:root:len(observation_set.GetObservations())=120
INFO:root:Unique counts of attributes per stage per well={2}
INFO:root:Wrote changes to "c:\src\Orchid.IntegrationTestData\frankNstein_Bakken_UTM13_FEET.998.ifrac"
"""

In [ ]:
@parsy.generate
def get_attribute_count_for_each_stage_and_well():
    yield project_name
    yield newline >> observation_set_items
    yield newline >> parent_well_observations
    yield newline >> get_observations
    yield newline >> auto_picked_observation_set
    yield newline >> get_observations
    attribute_count_for_each_stage_and_well = yield newline >> attribute_count_per_stage_per_well
    yield newline >> output_path_name
    yield newline

    return attribute_count_for_each_stage_and_well

In [ ]:
get_attribute_count_for_each_stage_and_well.parse(auto_pick_and_create_attributes_output_text)